In [ ]:
#using tutorial https://www.tensorflow.org/tutorials/text/text_generation#advanced_customized_training 

In [1]:
import pandas as pd
import numpy as np

#more imports
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding
from tensorflow.keras.layers import LSTM
from tensorflow.keras.optimizers import RMSprop

import random
import sys
import os
import time

import tensorflow as tf
import tensorflow_datasets as tfds

In [2]:
# Read in the data
df = pd.read_csv("ufo_update.csv")
text_ = " "

In [3]:
df.head()

,text,stats,date_time,report_link,city,state,shape,duration,summary,posted
0,The background is a blue sky. The shape of th...,Occurred : 8/5/2020 10:15 (Entered as : 08/05...,8/5/20 10:15,http://www.nuforc.org/webreports/158/S158217.html,Ty Ty,GA,Unknown,5 minutes,The background is a blue sky. The shape of the...,8/6/20
1,Black polygonal craft with lights wobbling thr...,Occurred : 8/5/2020 18:50 (Entered as : 08/05...,8/5/20 18:50,http://www.nuforc.org/webreports/158/S158226.html,New Braunfels,TX,Chevron,2-3 minutes,Black polygonal craft with lights wobbling thr...,8/6/20
2,I observed a white ball of light flying from t...,Occurred : 8/4/2020 20:30 (Entered as : 08/04...,8/4/20 20:30,http://www.nuforc.org/webreports/158/S158228.html,Mesa,AZ,Circle,2 minutes,I observed a white ball of light flying from t...,8/6/20
3,High altitude fast moving delta shaped flames ...,Occurred : 8/4/2020 22:00 (Entered as : 08/04...,8/4/20 22:00,http://www.nuforc.org/webreports/158/S158221.html,Miami Beach,FL,Fireball,1 minute,High altitude fast moving delta shaped flames....,8/6/20
4,Three unmistakable lights moving at a speed th...,Occurred : 8/5/2020 04:33 (Entered as : 08/05...,8/5/20 04:33,http://www.nuforc.org/webreports/158/S158220.html,Pheonix,AZ,Triangle,3 seconds,Three unmistakable lights moving at a speed th...,8/6/20


In [4]:
df['text'][0]

'The background is a blue sky.  The shape of the UFO is indescribable. UFO was moving from north to south at about tree top level at a very slow speed as if looking for something.'

In [5]:
df.shape

(43886, 10)

In [6]:
#sub = df.sample(frac=.5)

sub = df.iloc[:1000]

In [7]:
sub.head()

,text,stats,date_time,report_link,city,state,shape,duration,summary,posted
0,The background is a blue sky. The shape of th...,Occurred : 8/5/2020 10:15 (Entered as : 08/05...,8/5/20 10:15,http://www.nuforc.org/webreports/158/S158217.html,Ty Ty,GA,Unknown,5 minutes,The background is a blue sky. The shape of the...,8/6/20
1,Black polygonal craft with lights wobbling thr...,Occurred : 8/5/2020 18:50 (Entered as : 08/05...,8/5/20 18:50,http://www.nuforc.org/webreports/158/S158226.html,New Braunfels,TX,Chevron,2-3 minutes,Black polygonal craft with lights wobbling thr...,8/6/20
2,I observed a white ball of light flying from t...,Occurred : 8/4/2020 20:30 (Entered as : 08/04...,8/4/20 20:30,http://www.nuforc.org/webreports/158/S158228.html,Mesa,AZ,Circle,2 minutes,I observed a white ball of light flying from t...,8/6/20
3,High altitude fast moving delta shaped flames ...,Occurred : 8/4/2020 22:00 (Entered as : 08/04...,8/4/20 22:00,http://www.nuforc.org/webreports/158/S158221.html,Miami Beach,FL,Fireball,1 minute,High altitude fast moving delta shaped flames....,8/6/20
4,Three unmistakable lights moving at a speed th...,Occurred : 8/5/2020 04:33 (Entered as : 08/05...,8/5/20 04:33,http://www.nuforc.org/webreports/158/S158220.html,Pheonix,AZ,Triangle,3 seconds,Three unmistakable lights moving at a speed th...,8/6/20


In [8]:
sub['text'] = sub['text'].astype(str)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
# for i in sub['text']:
#   text_ += i

# text_ = text_.split(" ")
# text = []

# for i in text_:
#   text.append(i+" ")

In [9]:
text = ' '.join(sub['text'].tolist())

In [17]:
#Clean text

import re
from bs4 import BeautifulSoup

def cleanText(txt):
    txt = BeautifulSoup(txt).get_text()
    txt = txt.replace('\t', '')
    txt = txt.replace('\x9d', '')
    txt = txt.replace('\xa0', '')
    #remove all non letters from text
    txt = re.sub("[^A-Za-z0-9_.,!'/$]", " ", txt) 
    txt = txt.split()
    txt = " ".join(txt)

    return txt

In [18]:
text = cleanText(text)

In [27]:
# Encode the data as Chars

#get all unique chars
chars = sorted(list(set(text)))
print('unique characters', format(len(chars)))

unique characters 70


In [32]:
# char_indices = dict((c, i) for i, c in enumerate(chars))
# indices_char = dict((i, c) for i, c in enumerate(chars))

#create a mapping from unique characters to indices

char_indices = {u:i for i, u in enumerate(chars)}
indices_char = np.array(chars)

text_as_int = np.array([char_indices[c] for c in text])

In [33]:
print('{')
for char,_ in zip(char_indices, range(70)):
    print('  {:4s}: {:3d},'.format(repr(char), char_indices[char]))
print('  ...\n}')

{
  ' ' :   0,
  '!' :   1,
  '$' :   2,
  "'" :   3,
  ',' :   4,
  '.' :   5,
  '/' :   6,
  '0' :   7,
  '1' :   8,
  '2' :   9,
  '3' :  10,
  '4' :  11,
  '5' :  12,
  '6' :  13,
  '7' :  14,
  '8' :  15,
  '9' :  16,
  'A' :  17,
  'B' :  18,
  'C' :  19,
  'D' :  20,
  'E' :  21,
  'F' :  22,
  'G' :  23,
  'H' :  24,
  'I' :  25,
  'J' :  26,
  'K' :  27,
  'L' :  28,
  'M' :  29,
  'N' :  30,
  'O' :  31,
  'P' :  32,
  'Q' :  33,
  'R' :  34,
  'S' :  35,
  'T' :  36,
  'U' :  37,
  'V' :  38,
  'W' :  39,
  'X' :  40,
  'Y' :  41,
  'Z' :  42,
  '_' :  43,
  'a' :  44,
  'b' :  45,
  'c' :  46,
  'd' :  47,
  'e' :  48,
  'f' :  49,
  'g' :  50,
  'h' :  51,
  'i' :  52,
  'j' :  53,
  'k' :  54,
  'l' :  55,
  'm' :  56,
  'n' :  57,
  'o' :  58,
  'p' :  59,
  'q' :  60,
  'r' :  61,
  's' :  62,
  't' :  63,
  'u' :  64,
  'v' :  65,
  'w' :  66,
  'x' :  67,
  'y' :  68,
  'z' :  69,
  ...
}


In [ ]:
# Create the Sequence Data

# maxlen = 40   #40 crashes colab with a larger dataset
# steps = 3

# sentences = [] #X
# next_chars = [] #Y
    
# for i in range(0, len(text)- maxlen, steps):
#   sentences.append(text[i: i + maxlen])
#   next_chars.append(text[i + maxlen])

In [46]:
# The maximum length sentence you want for a single input in characters

seq_length = 100   #try adjusting sequence length
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(indices_char[i.numpy()])   #do not alias numpy as np

T
h
e
 
b


In [ ]:
# Specify X and y
# x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)

In [ ]:
# y = np.zeros((len(sentences), len(chars)), dtype=np.bool)

In [ ]:
# for i, sentence in enumerate(sentences):
#   for t, char in enumerate(sentence):
#     x[i, t, char_indices[char]] = 1
#   y[i, char_indices[next_chars[i]]] = 1

In [47]:
#The batch method lets us easily convert these individual characters to sequences of the desired size.

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
  print(repr(''.join(indices_char[item.numpy()])))  #do not alias numpy as np

'The background is a blue sky. The shape of the UFO is indescribable. UFO was moving from north to sou'
'th at about tree top level at a very slow speed as if looking for something. Black polygonal craft wi'
'th lights wobbling through sky. I was in the back yard area of my apartment complex with only my dog '
'when I saw what I thought was a drone. 1 solid black object that appeared to be a pentagonal pyramid '
'flying slowly through the air. It looked to be about the size of a small car and less than 1000ft or '


In [41]:
#For each sequence, duplicate and shift it to form the input and target text by 
#using the map method to apply a simple function to each batch:

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [45]:
#Print the first example input and target values:

for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(''.join(indices_char[input_example.numpy()])))
    print('Target data:', repr(''.join(indices_char[target_example.numpy()])))

Input data:  'The background is a blue sky. The shape of the UFO is indescribable. UFO was moving from north to so'
Target data: 'he background is a blue sky. The shape of the UFO is indescribable. UFO was moving from north to sou'


In [48]:
#each index of these vectors is processed as a one time step. For the input at
#time step 0, the model receives the index for 'T' and tries to predict the
#index for i as the next character. At the next timestep, it does the same thing
#but the RNN considers the previous step context in addition to the current input
#character

for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(indices_char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(indices_char[target_idx])))

Step    0
  input: 36 ('T')
  expected output: 51 ('h')
Step    1
  input: 51 ('h')
  expected output: 48 ('e')
Step    2
  input: 48 ('e')
  expected output: 0 (' ')
Step    3
  input: 0 (' ')
  expected output: 45 ('b')
Step    4
  input: 45 ('b')
  expected output: 44 ('a')


In [49]:
#Create training batches - you can use tf.data to split the text into manageable
#sequences. But before feeding this data into the model, you need to shuffle the
#data and pack it into batches

# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int64, tf.int64)>

Build the model
For each character the model looks up the embedding, runs the GRU one timestep
with the embedding as input, and applies the dense layer to generate logits
predicting the log-likelihood of the next character

In [111]:
# Length of the vocabulary in chars
vocab_size = len(chars)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [112]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [113]:
model = build_model(
    vocab_size=len(chars),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [114]:
#check the shape of the output

for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length,vocab_size)")

(64, 100, 70) # (batch_size, sequence_length,vocab_size)


In [115]:
model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (64, None, 256)           17920     
_________________________________________________________________
gru_6 (GRU)                  (64, None, 1024)          3938304   
_________________________________________________________________
dense_6 (Dense)              (64, None, 70)            71750     
Total params: 4,027,974
Trainable params: 4,027,974
Non-trainable params: 0
_________________________________________________________________


In [116]:
#to get prediction from the model you need to sample from the output 
#distribution to get actual character indices

sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

sampled_indices

array([13, 68, 66, 28, 15, 32, 22, 13, 41, 17,  7, 27, 32, 61,  7, 50, 35,
       12, 36, 49, 37, 53, 46, 69,  9, 14, 60,  7, 57, 20, 31,  1, 29,  5,
        8, 20, 55, 51, 55, 37, 47, 43,  6,  7, 62, 31, 16, 67,  5, 34, 23,
       57, 57, 20, 44,  6, 32, 34,  1, 64,  6, 34, 16, 41, 29, 44, 54, 56,
       68, 58, 56,  1,  1, 42, 13, 42, 67, 44, 39, 18, 20, 45,  5, 30,  3,
        7,  8, 33, 60,  4, 20, 18, 29, 48, 67, 42, 67, 11, 43,  6])

In [117]:
#decode these to see the text predicted by this untrained model

print("Input: \n", repr("".join(indices_char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(indices_char[sampled_indices ])))

Input: 
 'e track/path as the rest of the procession. There was no flashing lights typical, of aircraft or any'

Next Char Predictions: 
 "6ywL8PF6YA0KPr0gS5TfUjcz27q0nDO!M.1DlhlUd_/0sO9x.RGnnDa/PR!u/R9YMakmyom!!Z6ZxaWBDb.N'01Qq,DBMexZx4_/"


In [118]:
#train the model - at this point problem can be treated as a standard
#classification problem. Given the previous RNN state, and the input this time
#step, predict the class of the next character

def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 70)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.2486253


In [119]:
#Configure the training procedure using the tf.keras.Model.compile method. 
#Use tf.keras.optimizers.Adam with default arguments and the loss function.

model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

In [120]:
#Use a tf.keras.callbacks.ModelCheckpoint to ensure that checkpoints 
#are saved during training:

# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [121]:
EPOCHS = 30

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/30
97/97 [==============================] - 3s 27ms/step - loss: 2.8631 - accuracy: 0.2641
Epoch 2/30
97/97 [==============================] - 3s 26ms/step - loss: 2.0820 - accuracy: 0.4028
Epoch 3/30
97/97 [==============================] - 3s 26ms/step - loss: 1.7753 - accuracy: 0.4931
Epoch 4/30
97/97 [==============================] - 3s 26ms/step - loss: 1.5603 - accuracy: 0.5555
Epoch 5/30
97/97 [==============================] - 3s 27ms/step - loss: 1.4244 - accuracy: 0.5907
Epoch 6/30
97/97 [==============================] - 3s 26ms/step - loss: 1.3329 - accuracy: 0.6141
Epoch 7/30
97/97 [==============================] - 3s 27ms/step - loss: 1.2664 - accuracy: 0.6312
Epoch 8/30
97/97 [==============================] - 3s 26ms/step - loss: 1.2124 - accuracy: 0.6451
Epoch 9/30
97/97 [==============================] - 3s 27ms/step - loss: 1.1668 - accuracy: 0.6573
Epoch 10/30
97/97 [==============================] - 3s 27ms/step - loss: 1.1254 - accuracy: 0.6680
Epoch 11/

Generate text - to keep prediction step simple use a batch size of 1
because of the way the RNN state is passed from timestep to timestep, the 
model only accepts a fixed batch size once built. To run the model with a diff
batch size, you need to rebuild the model and restore the weights from the checkpoint. 

Looking at the genearated text, the model knows when to capitalize, make
paragraphs and imitate the vocabulary. With a small # of epochs it has not learned to form coherent sentences

In [122]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_30'

In [123]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [124]:
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (1, None, 256)            17920     
_________________________________________________________________
gru_7 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_7 (Dense)              (1, None, 70)             71750     
Total params: 4,027,974
Trainable params: 4,027,974
Non-trainable params: 0
_________________________________________________________________


The prediction loop: the following code generates the text.
begin by choosing a start string, initializing the RNN state and setting the 
number of characters to generate. Get the prediction distribution of the next
character using the start string and the RNN state. Then use a categorical 
distribution to calculate the index of the predicted character. Use this
predicted character as our next input to the model. The RNN state returned by 
the model is fed back into the model so that it now has more context, instead
only one character. After predicting the next character, the modified RNN states
are again fed back into the model, which is how it learns more context from the
previously predicted characters

In [125]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 1000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char_indices[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(indices_char[predicted_id])

    return (start_string + ''.join(text_generated))

In [127]:
print(generate_text(model, start_string=u"I saw a little green man "))

I saw a little green man made craft. We live in a flight path. They were heading from the south at an extreme spot in order. Seemed to be green to be just a long line. Very long line formation heading towards Mallowish Clear sky Chenomenon I've witnessed a short gray alien with a lasht red. he missed the craft watching the sknut lite one before it was a train that it went away further its sky and it was flying North. Blue going around the sphere.tthing the light disappear from my video is examined the lights as well as brightness. The line of lights in sky above where we entrese looked at the stars when i noticed the formation coming in from the location with what appears to be as bright as the tree line. I asked my gar was diagonally, but was time the light crossed the object. I didn t white on across with that area, such as I was driving. UFO sighting it was many more aircraft. There was a light breeze, and fleared in formation forming a triangle. My initial thought UFO sighting. Obj

Advanced: Customized Training
The above training procedure is simple, but does not give you much control.

So now that you've seen how to run the model manually let's unpack the training loop, and implement it ourselves. This gives a starting point if, for example, you want to implement curriculum learning to help stabilize the model's open-loop output.

Use tf.GradientTape to track the gradients. You can learn more about this approach by reading the eager execution guide.

The procedure works as follows:

First, reset the RNN state. You do this by calling the tf.keras.Model.reset_states method.

Next, iterate over the dataset (batch by batch) and calculate the predictions associated with each.

Open a tf.GradientTape, and calculate the predictions and loss in that context.

Calculate the gradients of the loss with respect to the model variables using the tf.GradientTape.grads method.

Finally, take a step downwards by using the optimizer's tf.train.Optimizer.apply_gradients method.

In [99]:
model = build_model(
    vocab_size=len(chars),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

In [100]:
optimizer = tf.keras.optimizers.Adam()

In [101]:
@tf.function
def train_step(inp, target):
    with tf.GradientTape() as tape:
        predictions = model(inp)
        loss = tf.reduce_mean(
            tf.keras.losses.sparse_categorical_crossentropy(
                target, predictions, from_logits=True))
    grads = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))

    return loss

In [102]:
# Training step
EPOCHS = 10

for epoch in range(EPOCHS):
    start = time.time()

    # resetting the hidden state at the start of every epoch
    model.reset_states()

    for (batch_n, (inp, target)) in enumerate(dataset):
        loss = train_step(inp, target)

        if batch_n % 100 == 0:
            template = 'Epoch {} Batch {} Loss {}'
            print(template.format(epoch + 1, batch_n, loss))

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print('Epoch {} Loss {:.4f}'.format(epoch + 1, loss))
    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

model.save_weights(checkpoint_prefix.format(epoch=epoch))

Epoch 1 Batch 0 Loss 4.248414039611816
Epoch 1 Loss 2.3068
Time taken for 1 epoch 3.9288082122802734 sec

Epoch 2 Batch 0 Loss 2.2624258995056152
Epoch 2 Loss 1.9295
Time taken for 1 epoch 2.8922977447509766 sec

Epoch 3 Batch 0 Loss 1.8997366428375244
Epoch 3 Loss 1.6693
Time taken for 1 epoch 2.9084513187408447 sec

Epoch 4 Batch 0 Loss 1.6758805513381958
Epoch 4 Loss 1.5015
Time taken for 1 epoch 2.8745174407958984 sec

Epoch 5 Batch 0 Loss 1.427892804145813
Epoch 5 Loss 1.4372
Time taken for 1 epoch 2.9053118228912354 sec

Epoch 6 Batch 0 Loss 1.3530628681182861
Epoch 6 Loss 1.3244
Time taken for 1 epoch 2.973132848739624 sec

Epoch 7 Batch 0 Loss 1.2630212306976318
Epoch 7 Loss 1.2531
Time taken for 1 epoch 2.9544434547424316 sec

Epoch 8 Batch 0 Loss 1.195586085319519
Epoch 8 Loss 1.2361
Time taken for 1 epoch 2.9149463176727295 sec

Epoch 9 Batch 0 Loss 1.180448293685913
Epoch 9 Loss 1.2096
Time taken for 1 epoch 2.967536211013794 sec

Epoch 10 Batch 0 Loss 1.0844241380691528
Ep

In [ ]:
# from keras.optimizers import RMSprop
# from keras.layers import Dense, Activation  #move later

# model = Sequential()
# model.add(LSTM(128, input_shape=(maxlen, len(chars))))
# model.add(Dense(len(chars)))
# model.add(Activation('softmax'))
# optimizer = RMSprop(lr=0.01)
# model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

In [ ]:
# def sample(preds, temperature=1.0):
#     # helper function to sample an index from a probability array
#     preds = np.asarray(preds).astype('float64')
#     preds = np.log(preds) / temperature
#     exp_preds = np.exp(preds)
#     preds = exp_preds / np.sum(exp_preds)
#     probas = np.random.multinomial(1, preds, 1)
#     return np.argmax(probas)

In [ ]:
# def on_epoch_end(epoch, _):
#     # Function invoked at end of each epoch. Prints generated text.
#     print()
#     print('----- Generating text after Epoch: %d' % epoch)

#     start_index = random.randint(0, len(text) - maxlen - 1)
#     for diversity in [0.2, 0.5, 1.0, 1.2]:
#         print('----- diversity:', diversity)

#         generated = ''
#         sentence = text[start_index: start_index + maxlen]
#         generated += sentence
#         print('----- Generating with seed: "' + sentence + '"')
#         sys.stdout.write(generated)

#         for i in range(400):
#             x_pred = np.zeros((1, maxlen, len(chars)))
#             for t, char in enumerate(sentence):
#                 x_pred[0, t, char_indices[char]] = 1.

#             preds = model.predict(x_pred, verbose=0)[0]
#             next_index = sample(preds, diversity)
#             next_char = indices_char[next_index]

#             generated += next_char
#             sentence = sentence[1:] + next_char

#             sys.stdout.write(next_char)
#             sys.stdout.flush()
#         print()

# print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [ ]:
# history_2 = model.fit(x, y,
#           batch_size=128,
#           epochs=100,
#           callbacks=[print_callback])

Epoch 1/100
1660/1662 [============================>.] - ETA: 0s - loss: 1.8400 - accuracy: 0.4849
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: " in a linear fashion, but much much high"
 in a linear fashion, but much much high and the same of the same from the sky and then the same blinked the same stars and I was a string and the same and the sky and the same stars and then the same the same stars and I was sound and the same of the sky and we have never stars and then stars the same and the same of the sky and the same and then a shore and the same and then a sat a streas and then a saw a straight line of the sky and
----- diversity: 0.5
----- Generating with seed: " in a linear fashion, but much much high"
 in a linear fashion, but much much high the sky and we went the same and the craft of the sky and I was 15 stars of the mitube and the sky in the circular all doing we place a carical of and the light seen and the light in the sky and for

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


to be a straight line and then disappeared and was a straight line and then disappeared to be a straight line and then all of a sudden and then disappeared and then disappeared to the south and they were all the same the other were moving for about 10 minutes.  It was a straight line and then they were at a star like the other moving from t
----- diversity: 0.5
----- Generating with seed: "n the edge of a field in Mequon WI. We w"
n the edge of a field in Mequon WI. We were moving or something like this they were all ship I then far object in the sky which was a satellite.  It was a stars.  They were all as I got and was a distance and after a military on the wester.  It was the same thing.  I was a bright light Lake around 5 minutes like a  I was it was a  The objects were low as it ascend approximatIly it started to see and was the binoculars then going outside
----- diversity: 1.0
----- Generating with seed: "n the edge of a field in Mequon WI. We w"
n the edge of a field in Mequon 

In [ ]:
# history_3 = model.fit(x, y,
#           batch_size=128,
#           epochs=150,
#           callbacks=[print_callback])

Epoch 1/150
1655/1662 [============================>.] - ETA: 0s - loss: 1.0963 - accuracy: 0.6811
----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "ht that kept catching my attention throu"
ht that kept catching my attention through the same speed and then the other was all the object was a flathes and the size of a slight line of an aircraft and the same direction of the same direction.  I was stars and then the other light was all the object was a flate object appeared to be a straight li

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


ne of a single of the sight and the size of a smalle star like a straight line and the same thing and then the other was a flate light 
----- diversity: 0.5
----- Generating with seed: "ht that kept catching my attention throu"
ht that kept catching my attention through the south to a second side. I was one of the part of the unidence and it was part of the line of the same thing I saw a bright light parts forming into the big diving from the right and the same thing was one by one I was along the lase and the other object on the west to east.  I saw what I saw, but it was a paperhored and then disappeared that I made the light appeared to be a line and there 
----- diversity: 1.0
----- Generating with seed: "ht that kept catching my attention throu"
ht that kept catching my attention through the way on the lights.  Nothing vestacject SE I destrated to find with my phone quickly go a formation of that the friend/leefsol of it. MADAR Node  This can't known that there was no pathin launc